# How to build an AI Agent with Microsoft Agent Framework and the ClickHouse MCP Server

## Install libraries

In [ ]:
!pip install agent-framework

In [4]:
from agent_framework import ChatAgent, MCPStdioTool
from agent_framework.openai import OpenAIResponsesClient

## ClickHouse MCP Server

In [5]:
clickhouse_mcp_server = MCPStdioTool(
    name="clickhouse",
    command="uv",
    args=[
        "run",
        "--with",
        "mcp-clickhouse",
        "--python",
        "3.10",
        "mcp-clickhouse"
    ],
    env={
        "CLICKHOUSE_HOST": "sql-clickhouse.clickhouse.com",
        "CLICKHOUSE_PORT": "8443",
        "CLICKHOUSE_USER": "demo",
        "CLICKHOUSE_PASSWORD": "",
        "CLICKHOUSE_SECURE": "true",
        "CLICKHOUSE_VERIFY": "true",
        "CLICKHOUSE_CONNECT_TIMEOUT": "30",
        "CLICKHOUSE_SEND_RECEIVE_TIMEOUT": "30"
    }
)

## Setup credentials
Let's provide our OpenAI API key.

In [6]:
import os, getpass

In [10]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key:")

Enter OpenAI API Key: ········


## Initialize and run agent

In [11]:
async with ChatAgent(
    chat_client=OpenAIResponsesClient(model_id="gpt-5-mini-2025-08-07"),
    name="DocsAgent",
    instructions="You are a helpful assistant that can help query a ClickHouse database",
    tools=clickhouse_mcp_server,
) as agent:
    query = "Tell me about UK property prices over the last five years"
    print(f"User: {query}")
    async for chunk in agent.run_stream(query):
        print(chunk.text, end="", flush=True)
    print("\n\n")

User: Tell me about UK property prices over the last five years
I queried the UK Price Paid dataset for the recent five-year period (using date >= addYears(today(), -5), i.e. roughly from 2020‑10‑06 through today). Summary findings, key numbers and observations below. If you want a different range (calendar years or last 5 full years) I can re-run.

High-level yearly summary (rough period: late‑2020 → 2025 YTD)
- Sales (counts): 2020: 306,857 | 2021: 1,279,406 | 2022: 1,073,122 | 2023: 854,549 | 2024: 859,960 | 2025 (YTD): 376,633
  - Note: 2021 shows a very large spike in transactions (policy/market effects such as the stamp‑duty holiday and COVID-era timing likely contributors).
- Average sale price (mean): 2020: £384,740 → 2021: £389,331 → 2022: £414,053 (peak) → 2023: £404,796 → 2024: £390,879 → 2025 (YTD): £362,283
  - Yearly changes (mean): +1.2% (20→21), +6.4% (21→22), −2.2% (22→23), −3.4% (23→24), −7.3% (24→25 YTD)
- Median sale price: 2020: £265,100 → 2021: £269,995 → 2022: £2